In [ ]:
!pip install langchain
!pip install openai

In [ ]:
%env SERPAPI_API_KEY=xxx
%env OPENAI_API_KEY=xxxxxx

## Conversational Retrieval QA

In [ ]:
!pip install faiss-cpu

### 文档资料
- 安牌官网（https://www.anpaitm.com/）
- 中华人民共和国商标法（https://www.wipo.int/wipolex/zh/text/547083）

In [ ]:
#for FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader


In [ ]:
#For FAISS
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.anpaitm.com/")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
#"我想让你担任安牌公司的商标顾问，你的名字叫做'媛姐助理'。我将描述商标相关的问题，您将提供有关如何处理的建议。除了商标相关的问题之外，你可以不回答其他的问题。安牌网的介绍：安牌，平台以大数据为基础、AI技术为驱动、智能产品为核心、专业服务为支撑，为企业提供从商标到品牌一站式的智慧化服务。推动大数据、人工智能技术在知识产权领域的深度应用，为企业商标到品牌保驾护航"
from langchain import PromptTemplate

# note that the input variables ('question', etc) are defaults, and can be changed

_template = """I want you to act as my brand advisor '媛姐AI'. I will describe a brand situation and you will provide advice on how to handle it. You should only reply with your advice, and nothing else. Do not write explanations.
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

condense_prompt = PromptTemplate.from_template(_template)

prompt_template = """I want you to act as my brand advisor '媛姐AI'. I will describe a brand situation and you will provide advice on how to handle it. You should only reply with your advice, and nothing else. Do not write explanations.
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""

combine_docs_custom_prompt = PromptTemplate.from_template(prompt_template)

qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0), 
    vectorstore.as_retriever(), # see below for vectorstore definition
    memory=memory,
    condense_question_prompt=condense_prompt,
    combine_docs_chain_kwargs=dict(prompt=combine_docs_custom_prompt),
    verbose=True
)

In [ ]:
query = "我想咨询一下，如果我要申请一个耐克的商标，需要走什么流程？"
result = qa({"question": query})

result["answer"]